In [139]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os 
import gzip
import random
import matplotlib.pyplot as plt
import math

In [140]:
# 获取文件并标记编号
root_file = 'E:\\JupyterNotebook\\pic_download_from_web\\'
filename = []
label = []
for i in range(len(os.listdir(root_file))):
    sub_file_name_dir = os.path.join(root_file, os.listdir(root_file)[i])
    for j in range(len(os.listdir(sub_file_name_dir))):
        sub_file_name = os.path.join(sub_file_name_dir,os.listdir(sub_file_name_dir)[j])
        filename.append(sub_file_name)
        label.append(i)
filenames = tf.constant(filename)
labels = tf.constant(label)

In [151]:
####################################################################
######### 1. 使用tf.data.Dataset实现Batch Generator    #############
####################################################################
# 从硬盘中读取图片
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.cond(tf.image.is_jpeg(image_string),lambda: tf.image.decode_jpeg(image_string), lambda: tf.image.decode_png(image_string))
    image_decoded = tf.image.convert_image_dtype(image_decoded, tf.float32)
    image_grayed = tf.image.rgb_to_grayscale(image_decoded)
    image_resized = tf.image.resize_images(image_grayed, [28, 28], method = 0)
    return image_resized, label
   
# 此时dataset中的一个元素是(filename, label) 
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels)) # 此时dataset中的一个元素是(image_resized, label) 
dataset = dataset.map(_parse_function) # 此时dataset中的一个元素是(image_resized_batch, label_batch) 

batch_size = 10
repeat_num = 10
buffer_sieze_num = 100
dataset = dataset.shuffle(buffer_size=buffer_sieze_num).batch(batch_size).repeat(repeat_num) # batch size=32, 读入磁盘图片和label,训练时重复10个epoch.

# image_pixels = 28*28

In [152]:
#  实现回归模型
x1 = tf.placeholder(tf.float32, shape=([None, 28,28,1]))
x = tf.reshape(x1, [-1, 784*1])
W = tf.Variable(tf.random_uniform([784*1, 10]))

b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x,W) + b)
# 训练模型
y_ = tf.placeholder('float', [None, 10])
cross_entropy = - tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
# 初始化模型参数
init = tf.global_variables_initializer()

In [154]:
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    sess.run(init)
    
    x_train, y_train = sess.run(next_element)
    y_train_onehot = sess.run(tf.one_hot(y_train, 10))
    sess.run(train_step, feed_dict={x1: x_train, y_:y_train_onehot })
#     while True:
#         try: 
#             x_train, y_train = sess.run(next_element)
#         except tf.errors.OutOfRangeError:
#             break
#     y_train_onehot = sess.run(tf.one_hot(y_train, 10))
#     sess.run(train_step, feed_dict={x1: x_train, y_:y_train_onehot })

In [332]:
# train_x_file = os.path.join(data_file, files[3])
# train_y_file = os.path.join(data_file, files[1])
# test_x_file = os.path.join(data_file, files[2])
# test_y_file = os.path.join(data_file, files[0])
# # print (test_x_file)
# with gzip.open(train_y_file, 'rb') as lbpath:
#     y_train = np.frombuffer(lbpath.read(), dtype = np.uint8, offset=8)
# with gzip.open(train_x_file, 'rb') as imgpath:
#     x_train = np.frombuffer(imgpath.read(), dtype = np.uint8, offset=16).reshape(len(y_train), 784)
# with gzip.open(test_y_file, 'rb') as lbpath:
#     y_test = np.frombuffer(lbpath.read(), dtype = np.uint8, offset=8)
# with gzip.open(test_x_file, 'rb') as imgpath:
#     x_test = np.frombuffer(imgpath.read(), dtype = np.uint8, offset=16).reshape(len(y_test),784)
# x_train = x_train.astype('float')
# y_train = y_train.astype('uint8')
# # print ((y_train))
# # y_train = tf.one_hot(y_train, 10)
# # with tf.Session() as sess:
# #     print (sess.run(y_train))
# x_test = x_test.astype('float')
# y_test = y_test.astype('uint8')

# #  实现回归模型
# x = tf.placeholder('float', [None, 784])
# W = tf.Variable(tf.random_uniform([784, 10]))
# b = tf.Variable(tf.zeros([10]))
# y = tf.nn.softmax(tf.matmul(x,W) + b)

# # 训练模型
# y_ = tf.placeholder('float', [None, 10])
# cross_entropy = - tf.reduce_sum(y_*tf.log(y))
# train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
# init  = tf.initialize_all_variables()

In [339]:
# for i in range(500):
# #     print ("Step:" , i+1)
#     train_index = np.floor(np.random.uniform(0,1,batch_size)*np.shape(x_train[:,0])).astype(int)
#     x_train_batch = x_train[train_index,:]
#     # print (y_train)
#     y_train_batch = y_train[train_index]
# #     print (y_train_batch)
#     with tf.Session() as sess:
#             sess.run(init)
#             output = sess.run(tf.one_hot(y_train_batch, 10))
#             sess.run(train_step, feed_dict = {x:x_train_batch, y_:output})

# 
# dataset = tf.data.Dataset.range(100)
# next_element = dataset.make_one_shot_iterator().get_next()
# with tf.Session() as sess:
#     for i in range(100):
#         value = sess.run(next_element)
#         print ('valeu is %5d' %value)


# 可初始化迭代器
# max_value = tf.placeholder(tf.int64, shape = [])
# dataset = tf.data.Dataset.range(max_value)
# iterator = dataset.make_initializable_iterator()
# next_element = iterator.get_next()

# with tf.Session() as sess:
#     sess.run(iterator.initializer, feed_dict={max_value:100})
#     for i in range(100):
#         value = sess.run(next_element)
#         print ('value is %5d' %value)
